In [2]:
import numpy as np
import xarray as xr
from collections import OrderedDict

In [ ]:
# from matplotlib import rc
# rc('text', usetex=True)
# rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 10})

In [10]:
def ds_fix_dims(ds):
    '''
    Fix time dimensions to be easier to call, put time in hours and put x and y in km
    '''
    if "time_fine" not in ds.variables:
        ds = ds.rename({str(u'time_series_4500_1800.0'): 'time_coarse', str(u'time_series_75_60.0'): 'time_fine', str(u'time_series_75_1800.0'): 'time_mid'})
    ds['time_coarse']=ds.time_coarse/3600
    ds['time_mid']=ds.time_mid/3600
    ds['time_fine']=ds.time_fine/3600
    ds['x'] = ds.x.astype(float)/(35)
    ds['y'] = ds.y.astype(float)/(35)
    return ds

def surf_pre_calc(ds):
    '''
    surface precip domain mean in mm and mean over the simulation time
    '''
    surface_precip_mean = ds.surface_precip_mean[fine_spin:]*1000
    surface_precip_mean = surface_precip_mean.mean()/(ds.time_fine[-1]-ds.time_fine[fine_spin])
    return surface_precip_mean

def surf_pre_calc_end(ds):
    '''
    surface precip domain mean in mm and mean over the simulation time
    '''
    if when=="end
    surface_precip_mean = ds.surface_precip_mean[-fine_ave:]*1000
    surface_precip_end_mean = surface_precip_mean.mean()/(ds.time_fine[-1]-ds.time_fine[fine_ave])
    return surface_precip_end_mean

def rwp_calc(ds):
    '''
    rwp domain mean in g m^(-2) and mean over the simulation time
    '''
    rwp_mean = ds.RWP_mean[fine_spin:]*1000
    rwp_mean = rwp_mean.mean()
    return rwp_mean

def rwp_calc_end(ds):
    '''
    rwp domain mean in g m^(-2) and mean over the simulation time
    '''
    rwp_mean = ds.RWP_mean[-fine_ave:]*1000
    rwp_end_mean = rwp_mean.mean()
    return rwp_end_mean

def rain(ds):
    rwp_mean = rwp_calc(ds)
    rwp_end_mean = rwp_calc_end(ds)
    sp_mean = surf_pre_calc(ds)
    sp_end_mean = surf_pre_calc_end(ds)
    return [rwp_mean.item(), rwp_end_mean.item(), sp_mean.item(), sp_end_mean.item()]

def tke_calc(ds): 
    tke_mean = ds.reske_mean + ds.subke_mean
    time_mean = ds.reske_mean[ds.reske_mean.dims[0]].values
    return tke_mean, time_mean

def lwp_total(ds):
    if include_spinup == True:
        lwp = ds.LWP_mean*1000
        time_arr = ds.time_fine
    else:
        lwp = ds.LWP_mean[fine_spin:]*1000
        time_arr = ds.time_fine[fine_spin:]
    
    lwp = lwp.where(lwp.values<1e20)
    lwp_last = lwp[last]
    lwp_mean = np.mean(lwp[-fine_ave:])
    lwp_tend = (lwp[last] - lwp[first])/(time_arr[last] - time_arr[first])
    lwp_teme = (np.mean(lwp[-fine_ave:]) - np.mean(lwp[:fine_ave]))/(time_arr[last] - time_arr[first])
    tendency = calc_tendency(lwp)
    return [lwp_last, lwp_mean, lwp_tend, lwp_teme], lwp.values, tendency, lwp[lwp.dims[0]].values

def lwp_smooth(ds):
    '''
    Moving average of the lwp and tendency
    '''
    if include_spinup == True:
        lwp = ds.LWP_mean*1000
        time_arr = ds.time_fine
    else:
        lwp = ds.LWP_mean[fine_spin:]*1000
        time_arr = ds.time_fine[fine_spin:]
    step = 55
    lwp_smooth = []
    lwp_tend = []
    lwp_tend_smooth = []
    i = 0
    for l in range(1, len(lwp)):
        lwp_tend.append((lwp[l] - lwp[l-1])/(lwp.time_fine[l] - lwp.time_fine[l-1]))
        if l>step-1:
            lwp_smooth.append(lwp[l-step:l].mean())
            if i>0:
                lwp_tend_smooth.append((lwp_smooth[i] - lwp_smooth[i-1])/(lwp.time_fine[l] - lwp.time_fine[l-1]))
                i+=1
    return [lwp_smooth[-1], np.mean(lwp_smooth[-55:]), lwp_tend[-1], np.mean(lwp_tend), (lwp_smooth[-1]-lwp_smooth[0])/(time_arr[last] - time_arr[first])]

def lwp_cloud_calc(lmmr, lwp):   
    cloudy_lwp = []
    t = []
    lwp_masked_arrs=[]
    for m in range(len(lmmr)):
        col_mask = layer_cloud_mask(lmmr, m)
        arr_mask = col_mask.values
        lwp_masked = lwp[m].where(arr_mask==1)
        lwp_masked_arrs.append(lwp_masked*1000)
        cloud_lwp_mean = lwp_masked.mean(axis=(0,1))
        cloudy_lwp.append(cloud_lwp_mean.item()*1000)
        t.append(lmmr[m][lmmr.dims[0]].item())

    lwp_masked = lwp_masked*1000
    return cloudy_lwp, t, lwp_masked, lwp_masked_arrs

def lwp_cloud(ds):
    if include_spinup == True:
        lwp = ds.lwp
        lmmr = ds.q_cloud_liquid_mass
        time_arr = ds.time_coarse
    else:
        lwp = ds.lwp[coarse_spin:]
        lmmr = ds.q_cloud_liquid_mass[coarse_spin:]
        time_arr = ds.time_coarse[coarse_spin:]
    
    cloudy_lwp, times, lwp_masked_last, lwp_masked_arrs = lwp_cloud_calc(lmmr, lwp)
    lwp_cloud_last = cloudy_lwp[last]
    lwp_cloud_mean = np.mean(cloudy_lwp[-coarse_ave:])
    lwp_cloud_tend = (cloudy_lwp[last]-cloudy_lwp[first])/(time_arr[last] - time_arr[first])
    lwp_cloud_teme = (np.mean(cloudy_lwp[-coarse_ave:])-np.mean(cloudy_lwp[:coarse_ave]))/(time_arr[last] - time_arr[first])
    tendency = calc_tendency(cloudy_lwp, times)
    return [lwp_cloud_last, lwp_cloud_mean, lwp_cloud_tend, lwp_cloud_teme],cloudy_lwp,tendency,times, lwp_masked_last, lwp_masked_arrs

def calc_tendency(dataarray, *times):
    tendency=[]
    if times:
        tseries = times[0]
    else:
        tseries = dataarray[dataarray.dims[0]].values
        dataarray = dataarray.values
    for t_ind in range(1, len(dataarray), 1):
        c_step = dataarray[t_ind]
        p_step = dataarray[t_ind - 1]
        dx = (c_step) - (p_step)
        t = tseries[t_ind] - tseries[t_ind-1]
        if t != 0:
            tendency.append(dx/t)
        else:
            tendency.append(dx/0.01)
    return tendency

def column_cloud_fraction(lmmr):
    cloud_frac=[]
    t =[]
    for m in range(len(lmmr)):
        col_mask = layer_cloud_mask(lmmr, m)
        total = col_mask.sum(axis=(0,1))
        f = total.item()/(250*250)
        cloud_frac.append(f)
        t.append(lmmr[m][lmmr.dims[0]].item())
    return cloud_frac, t

def clfrac(ds):
    if include_spinup == True:
        lmmr = ds.q_cloud_liquid_mass
        time_arr = ds.time_coarse
    else:
        lmmr = ds.q_cloud_liquid_mass[coarse_spin:]
        time_arr = ds.time_coarse[coarse_spin:]

    cloud_frac, times = column_cloud_fraction(lmmr)
    cloud_frac_last = cloud_frac[last]
    cloud_frac_mean = np.mean(cloud_frac[-coarse_ave:])
    cloud_frac_tend = (cloud_frac[last] - cloud_frac[first])/(time_arr[last] - time_arr[first])
    cloud_frac_teme = (np.mean(cloud_frac[-coarse_ave:]) - np.mean(cloud_frac[:coarse_ave]))/(time_arr[last] - time_arr[first])
    tendency = calc_tendency(cloud_frac, times)
    return [cloud_frac_last, cloud_frac_mean, cloud_frac_tend, cloud_frac_teme], cloud_frac, tendency, times

def layer_cloud_mask(dataarray, time):
    '''
    Applies mask to each timestep and sums
    '''
    for n in range(110):
        layer = dataarray[time,:,:,n]
        dataarray[time,:,:,n] = layer.where(layer.values<1e-5,1).where(layer.values>1e-5,0)
    col_sum = dataarray[time].sum(axis=2,skipna=True)
    col_mask = col_sum.where(col_sum.values<1,1)
    return col_mask

In [11]:
### Load design and set paths to data
nd = "low_nd"  # "low_nd" or "high_nd"
ppe_path   = "/gws/nopw/j04/carisma/eers/dycoms/{}/PPE/ppe".format(nd)
val_path   = "/gws/nopw/j04/carisma/eers/dycoms/{}/VAL/val".format(nd)
extra_path = "/gws/nopw/j04/carisma/eers/dycoms/{}/EXTRA/extra".format(nd)
base_path  = "/gws/nopw/j04/carisma/eers/dycoms/{}/BASE/base/base.nc".format(nd)
design       = np.loadtxt("/home/users/eers/dycoms/designs/EmulatorInputsDesign2D.csv",delimiter=",", skiprows=1)
validation   = np.loadtxt("/home/users/eers/dycoms/designs/ValidationInputsDesign2D.csv", delimiter=",", skiprows=1)
extra_design = np.loadtxt("/home/users/eers/dycoms/designs/extra_points.csv",delimiter=",")
base = np.array([[-7.5, 8.5]])

ppe_no   = 20
val_no   = 8
extra_no = 6
base_no  = 1

In [12]:
od = OrderedDict()
od["ppe"]   = [ppe_no, ppe_path, design]
od["val"]   = [val_no, val_path, validation]
od["extra"] = [extra_no, extra_path, extra_design]
od["base"]  = [base_no, base_path, base]

### Initialise np arrays ###
last = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))
mean = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))
tend = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))
teme = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))

arrays = [last, mean, tend, teme]

rwp     = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))
rwp_end = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))
surface_precip     = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))
surface_precip_end = np.empty((ppe_no+oat_no+val_no+extra_no+base_no, 3))

rain_arrays = [rwp, rwp_end, surface_precip, surface_precip_end]

In [13]:
### Options ###
include_spinup = False # See fine_spin and coarse_spin for setup

first       = 0
last        = -1
coarse_spin = 2   # 1 hour = 1, 2 hours = 2
fine_spin   = 111 # 1 hour = 55, 2 hours = 111
coarse_ave  = 3
fine_ave    = 111  

### Loop through PPE datasets ###
calc    = 'lwp_cloud'      # options: 'lwp_cloud', 'cloud_frac', 'rain'
type_nd = "lwp_low"  # [lwp/cf/rain]_[low/high] but do lwp_total_low/high if doing lwp_total.
i=0

# lines=[]
# lwp_scenes=[]
type_timeseries=[]

for key in od:
    for j in range(od[key][0]):
        k = j+1
        nc = od[key][1] + str(k) + "/" + key + str(k) + ".nc"
        if key=='base':
            nc = base_path 
        ds = xr.open_dataset(nc)
        ds = ds_fix_dims(ds)
            
        if calc == 'lwp_cloud':
            output_array, timeseries, tendency, times, lwp_masked_last, lwp_masked_arrs = lwp_cloud(ds)
            time_hrs = times
            for step, masked_arr in enumerate(lwp_masked_arrs):
                np.savetxt(f"{key}_scenes/scene_{step}.csv",masked_arr.values, delimiter=',')

            ### uncomment to save top-down scenes
            # np.savetxt("lwp_scenes_last/lnd_lwp_scene_last_%s%s.csv"%(key,k), lwp_masked_last.values, delimiter=',')
        elif calc == 'cloud_frac':
    	    output_array, timeseries, tendency, times = clfrac(ds)
    	    time_hrs = times
        elif calc == 'rain':
    	    output_array = rain(ds)
        else:
            print('Select calc')
            break

        if calc in ['cloud_frac','lwp_cloud']:
    	    for b, array in enumerate(arrays):
                array[i, 0] = od[key][2][j][1]
                array[i, 1] = od[key][2][j][0]
                array[i, 2] = output_array[b]
        elif calc == 'rain':
    	    for b, array in enumerate(rain_arrays):
    	        array[i, 0] = od[key][2][j][1]
    	        array[i, 1] = od[key][2][j][0]
    	        array[i, 2] = output_array[b]

        ### uncomment to save ti
        # if key=='ppe':
        #     type_timeseries.append(timeseries)
        # elif key=='base':
        #     base=timeseries
        # np.savetxt('./timeseries/{}/{}{}_timeseries.csv'.format(type_nd,key,str(k)),timeseries,delimiter=',')   # need type_nd instead of calc if not doing lwp_cloud
        # np.savetxt('./timeseries/{}/{}{}_times.csv'.format(type_nd,key,str(k)),time_hrs,delimiter=',')
        ds.close()
        print(i)
        i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [15]:
if calc == 'rain':
    np.savetxt("dycoms_data_{0}_{1}_rwp.csv".format(nd, calc), rain_arrays[0], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_rwp_end.csv".format(nd, calc), rain_arrays[1], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_surfpre.csv".format(nd,calc), rain_arrays[2], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_surfpre_end.csv".format(nd,calc), rain_arrays[3], delimiter=",")
elif calc == 'lwp_smooth':
    np.savetxt("dycoms_data_{0}_{1}_lwp_last.csv".format(nd,calc), testing_arrays[0], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_lwp_mean_lasthr.csv".format(nd,calc), testing_arrays[1], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_lwp_tend_last.csv".format(nd,calc), testing_arrays[2], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_lwp_tend_ave.csv".format(nd,calc), testing_arrays[3], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_lwp_tend_diff.csv".format(nd,calc), testing_arrays[4], delimiter=",")
else:
    print("Saving..")
    np.savetxt("dycoms_data_{0}_{1}_last.csv".format(nd,calc), arrays[0], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_mean.csv".format(nd,calc), arrays[1], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_tend.csv".format(nd,calc), arrays[2], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_teme.csv".format(nd,calc), arrays[3], delimiter=",")
#np.savetxt("ppe_{}_timeseries.csv".format(type_nd), type_timeseries, fmt='%s', delimiter=',')

In [ ]:
### Variability ###
'''
var_path = "/gws/nopw/j04/carisma/eers/dycoms_sim/{}INT_VAR/".format(nd)
points = {"thick":"Highest LWP","kparam":"On $\kappa$ line", "thin":"Lowest LWP"}
equiv = ["ppe18", "ppe19", "ppe20"]
### Create lists ###
e_last=[]
e_mean=[]
e_tend=[]
e_teme=[]
lines=[]
i=0

for point in points:
    if point=='kparam':
        colour = (238/255, 27/255, 155/255)
        original = equiv[2]
        tmp_run = 5
    elif point=='thick':
        colour = (255/255, 211/255, 29/255)
        original = equiv[1]
        tmp_run = 2
    elif point=='thin':
        colour = (26/255, 224/255, 203/255)
        original = equiv[0]
        tmp_run = 5

    for i in range(tmp_run):# dropped to 4 extras + the 1 original run
        if i==(tmp_run-1):
            ds=xr.open_dataset("/gws/nopw/j04/carisma/eers/dycoms_sim/{0}PPE/{1}/{1}.nc".format(nd,original))
        else:
            name = point + str(i+1)
            nc = var_path + point + "/" + name + "/" + name + ".nc" 
            ds = xr.open_dataset(nc)

        ds = ds_fix_dims(ds)
        hours = 8

        if calc == 'lwp_total':
            output_array, timeseries, tendency, times = lwp_total(ds, hours)
            time_hrs = times
        elif calc == 'lwp_cloud':
            output_array, timeseries, tendency, times,lwp_masked = lwp_cloud(ds, hours)
            time_hrs = times
            #np.savetxt("lwp_scenes_values_intvar_%s.csv"%(name), lwp_masked.values, delimiter=',')
        elif calc == 'cloud_frac':
            output_array, timeseries, tendency, times = clfrac(ds, hours)
            time_hrs = times
        else:
            print("Select calc")
            break

        #np.savetxt('./timeseries/{}/{}{}_timeseries.csv'.format(type_nd,point,str(i+1)), timeseries,delimiter=',')
        #np.savetxt('./timeseries/{}/{}{}_times.csv'.format(type_nd,point,str(i+1)),time_hrs,delimiter=',')

        e_last.append(output_array[0])
        e_mean.append(output_array[1])
        e_tend.append(output_array[2])
        e_teme.append(output_array[3])
    print(point + ' finished')    

output_type=e_mean
#thick_mean = "{:.2f}".format(np.mean(output_type[0:4]))
#kparam_mean = "{:.2f}".format(np.mean(output_type[5:9]))
#thin_mean = "{:.2f}".format(np.mean(output_type[10:14]))
#thick_var = "{:.2f}".format(np.var(output_type[0:4]))
#kparam_var = "{:.2f}".format(np.var(output_type[5:9]))
#thin_var ="{:.2f}".format(np.var(output_type[10:14]))
#labels = "%s, $\mu = %s,\; \sigma^{2} = %s$"

#np.savetxt('ensemble_%s.csv'%calc, [e_last, e_mean, e_tend, e_teme], delimiter=',')
'''

In [7]:
#calc='cloud_frac'
calc='lwp_cloud'

type_nd = "lwp_cloud_low"
### Variability ###
var_path = "/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/" #.format(nd)
points = ['ppe3','ppe9','ppe11','ppe14','ppe15','ppe17','ppe18','ppe19','ppe20']

### Create lists ###
e_last=[]
e_mean=[]
e_tend=[]
e_teme=[]
lines=[]
i=0
for point in points:
    for i in range(1,6):# dropped to 4 extras + the 1 original run
        if i==(5):
            ds=xr.open_dataset(f"/gws/nopw/j04/carisma/eers/dycoms/low_nd/PPE/{point}/{point}.nc")
            print('ds_last = original')
        else:
            name = '{0}_{1}'.format(point, i)
            nc = '{0}{1}/{2}.nc'.format(var_path,point,name)
            print(nc)
            ds = xr.open_dataset(nc)

        ds = ds_fix_dims(ds)
        #hours = 8

        if calc == 'lwp_total':
            output_array, timeseries, tendency, times = lwp_total(ds)
            time_hrs = times   
            e_mean.append(output_array[1])
            e_teme.append(output_array[3])
        elif calc == 'lwp_cloud':
            output_array, timeseries, tendency, times, lwp_masked_last, lwp_diff = lwp_cloud(ds)
            time_hrs = times
            np.savetxt("lwp_scenes_last/lnd_lwp_scene_last_%s%s.csv"%(point,i),lwp_masked_last.values, delimiter=',')
            e_mean.append(output_array[1])
            e_teme.append(output_array[3])
        elif calc == 'cloud_frac':
            output_array, timeseries, tendency, times = clfrac(ds)
            time_hrs = times
            #e_last.append(output_array[0])
            #e_tend.append(output_array[2])
            e_mean.append(output_array[1])
            e_teme.append(output_array[3])
        else:
            print("Select calc")
            break

        #np.savetxt('./timeseries/{}/{}{}_timeseries.csv'.format(type_nd,point,str(i+1)), timeseries,delimiter=',')
        #np.savetxt('./timeseries/{}/{}{}_times.csv'.format(type_nd,point,str(i+1)),time_hrs,delimiter=',')

    print(point + ' finished')    

if calc=='lwp_total':
    out_list = [e_mean, e_teme]
elif calc=='cloud_frac':
    #out_list = [e_last, e_tend]
    out_list = [e_mean, e_teme]
elif calc=='lwp_cloud':
    out_list = [e_mean,e_teme]
out_arr_T = np.array(out_list).T
np.savetxt('ensemble_{}_mean.csv'.format(calc), out_arr_T, delimiter=',')


/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe3/ppe3_1.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe3/ppe3_2.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe3/ppe3_3.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe3/ppe3_4.nc
ds_last = original
ppe3 finished
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe9/ppe9_1.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe9/ppe9_2.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe9/ppe9_3.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe9/ppe9_4.nc
ds_last = original
ppe9 finished
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe11/ppe11_1.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe11/ppe11_2.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe11/ppe11_3.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe11/ppe11_4.nc
ds_last = original
ppe11 finished
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe14/ppe14_1.nc
/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/ppe14/ppe

In [31]:
out_list = [e_mean, e_teme]
out_arr_T = np.array(out_list).T
np.savetxt('ensemble_{}.csv'.format(calc), out_arr_T, delimiter=',')